In [ ]:
import sys
import numpy as np
import os
import glob
import pandas as pd
from plotnine.stats.stat_boxplot import weighted_percentile
import matplotlib.pyplot as plt
from onstove import MCA, OnStove, RasterLayer, VectorLayer
from helper_functions import available_layers, histogram, prioritize_households, prioritize_households_2

In [ ]:
snake = True  # Change this to False to run the notebook by yourself and not through snakemake

In [ ]:
if snake:
    output_directory =  snakemake.params.mca_folder
    results_directory = snakemake.params.onstove_folder
    gis_data = snakemake.params.processed_data
else:
    output_directory =  os.path.join('..', '4. Results', 'MCA')
    results_directory = os.path.join('..', '4. Results', 'Stated_policies')
    gis_data = os.path.join('..', '4. Results', 'Processed GIS Data')

# 1. Creating an MCA model

In [ ]:
nepal = MCA(project_crs=3857, cell_size=(1000, 1000))
nepal.output_directory = output_directory

In [ ]:
adm_path = os.path.join(gis_data, 'Administrative', 'Country_boundaries', 'Country_boundaries.geojson')
nepal.add_mask_layer(category='Administrative', name='Country_boundaries', path=adm_path)

## 1.1. Adding data layers
Similar to the data processing steps, we can add data layers using the `add_layer` method. A layer `category`, `name` and `path` also need to be provided. In addition, other arguments can be passed:
* `category`: this is used to group all datasets into a category in the final output, e.g. `demand` or `supply`. 
* `name`: the name of the layer.
* `path`: relative path where the layer file is stored, including the name and extension.
* `layer_type`: this argument is required with two possible options `raster` or `vector`, we should pass either one according to the dataset you are adding. 
* `scoring_method`: the method to be used to calculate the distance raster for this layer. 
    * For `raster` layers, only `log` is available. This will create a logarithmic representation of the original raster. If this is not desired, we should pass `None` to the `distance` parameter (this is the defaul value), which will take the original raster as the distance raster.
    * For `vector` layers, two options are available `proximity` and `time_travel`. `proximity`, calculates a distance raster using the vectors (e.g. points, lines, polygons) as origin; the output raster will have the same dimentions and cell size as the `base_layer` (i.e. see below). `time_travel`, will need a friction raster (which can be added later) that will be used to calculate the least cost path to every cell taking the vectors as starting points.
* `normalization`: currently only `MinMax` normalization is available. This is used to normalize the distance rasters in a 0 to 1 range, which will be used later to calculate the CCE Demand, Supply and Clean Cooking potential indexes.
* `inverse`: we can specify if the scale of the ormalization need to be reversed, depending on the nature of the dataset.

#### We will then read in the layers that we created in the `Data Processing` step:

In [ ]:
nepal.add_layer(category='Demographics', name='Population', 
                path=os.path.join(gis_data, 'Demographics', 'Population', 'Population.tif'),
                layer_type='raster', normalization='MinMax', inverse=False, 
                distance_method='log', base_layer=True)

nepal.add_layer(category='Electricity', name='Existing MV lines', 
                path=os.path.join(gis_data, 'Electricity', 'MV_lines', 'MV_lines.geojson'),
                layer_type='vector',
                normalization='MinMax', inverse=True, 
                distance_method='proximity')

nepal.add_layer(category='Electricity', name='Existing Mini Grids', 
                path=os.path.join(gis_data, 'Electricity', 'MG_points', 'MG_points.geojson'),
                layer_type='vector',
                normalization='MinMax', inverse=True, 
                distance_method='proximity')

nepal.add_layer(category='LPG', name='Traveltime', 
                path=os.path.join(gis_data, 'LPG', 'LPG_traveltime', 'LPG_traveltime.tif'),
                layer_type='raster', normalization='MinMax')

nepal.add_layer(category='Other', name='Roads', 
                path=os.path.join(gis_data, 'LPG', 'Roads', 'Roads.geojson'),
                layer_type='vector', 
                normalization='MinMax', inverse=True, 
                distance_method='proximity')

nepal.add_layer(category='Biomass', name='Canopy cover', 
                path=os.path.join(gis_data, 'Biomass', 'Forest', 'Forest.tif'), 
                normalization='MinMax',
                layer_type='raster')

nepal.add_layer(category='Biomass', name='Traveltime', 
                path=os.path.join(gis_data, 'Biomass', 'Friction', 'Friction.tif'), 
                normalization='MinMax',
                distance_method='travel_time',
                layer_type='raster')
nepal.layers['Biomass']['Traveltime'].starting_points = nepal.layers['Biomass']['Canopy cover'].start_points(lambda x: x >= 30)

nepal.add_layer(category='Demographics', name='Wealth', 
                path=os.path.join(gis_data, 'Demographics', 'Wealth', 'Wealth.tif'),
                layer_type='raster', normalization='MinMax', inverse=False)

#### We can also read layers from the `OnStove` cost-benefit analysis:

In [ ]:
scenario = 'Stated_policies'

files = glob.glob(os.path.join(results_directory, 'Rasters', '*.tif'))

for file in files:
    name = os.path.splitext(os.path.basename(file))[0]
    nepal.add_layer(category='OnStove', name=name, 
                    path=file,
                    layer_type='raster', normalization='MinMax')

## 1.2. Calculate distance rasters

In [ ]:
nepal.get_distance_rasters(datasets='all')

In [ ]:
nepal.layers['Electricity']['Existing infra'] = RasterLayer(category='Electricity', name='Existing infra')
nepal.layers['Electricity']['Existing infra'].data = np.maximum(nepal.layers['Electricity']['Existing MV lines'].distance_raster.data, 
                                                                   nepal.layers['Electricity']['Existing Mini Grids'].distance_raster.data)
nepal.layers['Electricity']['Existing infra'].meta = nepal.layers['Electricity']['Existing MV lines'].distance_raster.meta
nepal.layers['Electricity']['Existing infra'].distance_raster = nepal.layers['Electricity']['Existing infra']

## 1.3. Save the model

In [ ]:
nepal.to_pickle('mca.pkl')

# 2. MCA analysis
## 2.1. Read the MCA model

In [ ]:
nepal = MCA.read_model(os.path.join(output_directory, 'mca.pkl'))

In [ ]:
available_layers(nepal.layers, indent=0)

In [ ]:
cmap = {"Biomass ICS T3": '#57365A', "LPG": '#40bdbc', "Traditional Biomass": '#673139',
        "Biogas": '#73af48', "Biogas and Biomass ICS T2": "#F6029E", "Biomass ICS T2": '#B6195E',
        "Biogas and LPG": "#156e5e",  "Biogas and Traditional Biomass": "#266AA6",
        "Biogas and Biomass ICS T3": "#bd57b6", "Biogas and Electricity": "#484673",
        "Electricity": '#d14a36', "Electricity and Biomass ICS T2": "#4D7126",
        "Electricity and LPG": "#dec610", "Electricity and Traditional Biomass": "#FFC107",
        "Electricity and Biomass ICS T3": "#1E88E5", "Electricity and Biogas": "#e17c05",
        "Mini Grids": "#ff7678", "Electricity and Mini Grids": "#ffd9df",
        "Biogas and Mini Grids": "#ffd9df"}

labels = {"Biogas and Electricity": "Electricity and Biogas"}

scale_bar_prop = dict(size=100000, style='double', textprops=dict(size=11), location=(0.5, 0.02),
                      linekw=dict(lw=1, color='black'), extent=0.01, loc='lower center')
north_arrow_prop = dict(size=30, location=(0.92, 0.92), linewidth=0.5)

nepal_onstove = OnStove.read_model(os.path.join(results_directory, 'results.pkl'))

## 2.2. Create the MCA scenarios
### 2.2.1. Biomass ICS Tier 3 stoves
By 2025, install 500,000 improved cookstoves, specifically in rural areas.

In [ ]:
nepal.layers['Electricity']['Existing infra'].inverse = False
nepal.layers['LPG']['Traveltime'].inverse = False
nepal.layers['Biomass']['Traveltime'].inverse = True
nepal.layers['Demographics']['Wealth'].inverse = True
nepal.layers['Demographics']['Population'].inverse = False
nepal.layers['OnStove']['maximum_net_benefit_per_household'].inverse = False
nepal.layers['OnStove']['available_biogas_mean'].inverse = True

nepal.layers['Electricity']['Existing infra'].distance_limit = None
nepal.layers['LPG']['Traveltime'].distance_limit = None
nepal.layers['Biomass']['Traveltime'].distance_limit = None
nepal.layers['Demographics']['Wealth'].distance_limit = None
nepal.layers['Demographics']['Population'].distance_limit = lambda x: x>0
nepal.layers['OnStove']['maximum_net_benefit_per_household'].distance_limit = None
nepal.layers['OnStove']['available_biogas_mean'].distance_limit = None

nepal.layers['Electricity']['Existing infra'].weight = 2.7
nepal.layers['LPG']['Traveltime'].weight = 3.3
nepal.layers['Biomass']['Traveltime'].weight = 4.3
nepal.layers['Demographics']['Wealth'].weight = 4.6
nepal.layers['Demographics']['Population'].weight = 2
nepal.layers['OnStove']['maximum_net_benefit_per_household'].weight = 4.1
nepal.layers['OnStove']['available_biogas_mean'].weight = 3.3

biomass_ics_index = nepal.get_index(datasets={
                                              'Demographics': ['Population', 'Wealth'],
                                              'Electricity': ['Existing infra'],
                                              'Biomass': ['Traveltime'],
                                              'LPG': ['Traveltime'],
                                              'OnStove': ['maximum_net_benefit_per_household',
                                                          'available_biogas_mean']
                                             }, 
                                    buffer=True, name='Biomass ICS T3')
biomass_ics_index.name = 'Biomass_ICS_T3_priority_areas'
biomass_ics_index.plot(cmap='magma', ticks=[0, 0.5, 1], tick_labels=['Low', 'Medium', 'High'],
                       admin_layer=nepal.mask_layer, figsize=(16,9),
                       scale_bar=scale_bar_prop, 
                       north_arrow=north_arrow_prop,
                       colorbar_kwargs=dict(orientation='horizontal', x=0.05, y=0.1, width=0.3, height=0.05,
                                            title_prop=dict(loc='left', labelpad=-55, fontweight='normal'))
                       )
biomass_ics_index.save_image(name=os.path.join(nepal.output_directory, 'Biomass_ICS_T3_priority_areas.pdf'), 
                             cmap='magma', ticks=[0, 0.5, 1], tick_labels=['Low', 'Medium', 'High'],
                             admin_layer=nepal.mask_layer, figsize=(13,9),
                             scale_bar=scale_bar_prop, 
                             north_arrow=north_arrow_prop,
                             colorbar_kwargs=dict(orientation='horizontal', x=0.05, y=0.1, width=0.3, height=0.05,
                                                  title_prop=dict(loc='left', labelpad=-55, fontweight='normal')))

In [ ]:
histogram(model=nepal_onstove, index=biomass_ics_index, name='Biomass ICS T3', 
          labels=labels, cmap=cmap, output_folder=nepal.output_directory)

### 2.2.2. Biogas digesters and stoves
By 2025, install an additional 200,000 household biogas plants.

In [ ]:
nepal.layers['Electricity']['Existing infra'].inverse = False
nepal.layers['LPG']['Traveltime'].inverse = False
nepal.layers['Biomass']['Traveltime'].inverse = False
nepal.layers['Demographics']['Wealth'].inverse = False
nepal.layers['Demographics']['Population'].inverse = True
nepal.layers['OnStove']['maximum_net_benefit_per_household'].inverse = False
nepal.layers['OnStove']['available_biogas_mean'].inverse = False

nepal.layers['Electricity']['Existing infra'].distance_limit = None
nepal.layers['LPG']['Traveltime'].distance_limit = None
nepal.layers['Biomass']['Traveltime'].distance_limit = None
nepal.layers['Demographics']['Wealth'].distance_limit = None
nepal.layers['Demographics']['Population'].distance_limit = lambda x: x>0
nepal.layers['OnStove']['maximum_net_benefit_per_household'].distance_limit = None
nepal.layers['OnStove']['available_biogas_mean'].distance_limit = None

nepal.layers['Electricity']['Existing infra'].weight = 3
nepal.layers['LPG']['Traveltime'].weight = 3.5
nepal.layers['Biomass']['Traveltime'].weight = 2.7
nepal.layers['Demographics']['Wealth'].weight = 4
nepal.layers['Demographics']['Population'].weight = 3.3
nepal.layers['OnStove']['maximum_net_benefit_per_household'].weight = 4.2
nepal.layers['OnStove']['available_biogas_mean'].weight = 4.3

biogas_index = nepal.get_index(datasets={'Demographics': ['Population', 'Wealth'],
                                         'Electricity': ['Existing infra'],
                                         'Biomass': ['Traveltime'],
                                         'LPG': ['Traveltime'],
                                         'OnStove': ['maximum_net_benefit_per_household',
                                                     'available_biogas_mean']}, 
                               buffer=True, name='Biogas')
biogas_index.name = 'Biogas_priority_areas'
biogas_index.plot(cmap='magma', ticks=[0, 0.5, 1], tick_labels=['Low', 'Medium', 'High'],
                  admin_layer=nepal.mask_layer, figsize=(16,9),
                  scale_bar=scale_bar_prop, north_arrow=north_arrow_prop,
                  colorbar_kwargs=dict(orientation='horizontal', x=0.05, y=0.1, width=0.3, height=0.05,
                                            title_prop=dict(loc='left', labelpad=-55, fontweight='normal')))
biogas_index.save_image(os.path.join(nepal.output_directory, 'Biogas_priority_areas.pdf'), 
                        cmap='magma', ticks=[0, 0.5, 1], tick_labels=['Low', 'Medium', 'High'],
                        admin_layer=nepal.mask_layer, figsize=(13,9), 
                        scale_bar=scale_bar_prop, north_arrow=north_arrow_prop,
                        colorbar_kwargs=dict(orientation='horizontal', x=0.05, y=0.1, width=0.3, height=0.05,
                                            title_prop=dict(loc='left', labelpad=-55, fontweight='normal')))

In [ ]:
histogram(model=nepal_onstove, index=biogas_index, name='Biogas', 
          labels=labels, cmap=cmap, output_folder=nepal.output_directory)

### 2.2.3. Induction electric cookstoves
By 2030, ensure 25% of households use electric stoves as their primary mode of cooking

In [ ]:
nepal.layers['Electricity']['Existing infra'].inverse = True
nepal.layers['LPG']['Traveltime'].inverse = False
nepal.layers['Biomass']['Traveltime'].inverse = False
nepal.layers['Demographics']['Wealth'].inverse = False
nepal.layers['Demographics']['Population'].inverse = False
nepal.layers['OnStove']['maximum_net_benefit_per_household'].inverse = False
nepal.layers['OnStove']['available_biogas_mean'].inverse = True

nepal.layers['Electricity']['Existing infra'].distance_limit = None
nepal.layers['LPG']['Traveltime'].distance_limit = None
nepal.layers['Biomass']['Traveltime'].distance_limit = None
nepal.layers['Demographics']['Wealth'].distance_limit = None
nepal.layers['Demographics']['Population'].distance_limit = lambda x: x>0
nepal.layers['OnStove']['maximum_net_benefit_per_household'].distance_limit = None
nepal.layers['OnStove']['available_biogas_mean'].distance_limit = None

nepal.layers['Electricity']['Existing infra'].weight = 4.6
nepal.layers['LPG']['Traveltime'].weight = 3.8
nepal.layers['Biomass']['Traveltime'].weight = 2.3
nepal.layers['Demographics']['Wealth'].weight = 4.3
nepal.layers['Demographics']['Population'].weight = 3.5
nepal.layers['OnStove']['maximum_net_benefit_per_household'].weight = 4.4
nepal.layers['OnStove']['available_biogas_mean'].weight = 2.3

electricity_index = nepal.get_index(datasets={'Demographics': ['Population', 'Wealth'],
                                       'Electricity': ['Existing infra'],
                                       'Biomass': ['Traveltime'],
                                       'LPG': ['Traveltime'],
                                       'OnStove': ['maximum_net_benefit_per_household',
                                                   'available_biogas_mean']}, 
                             buffer=True, name='Electricity')
electricity_index.name = 'Electricity_priority_areas'
electricity_index.plot(cmap='magma', ticks=[0, 0.5, 1], tick_labels=['Low', 'Medium', 'High'],
                       admin_layer=nepal.mask_layer, figsize=(16,9),
                       scale_bar=scale_bar_prop, north_arrow=north_arrow_prop,
                       colorbar_kwargs=dict(orientation='horizontal', x=0.05, y=0.1, width=0.3, height=0.05,
                                            title_prop=dict(loc='left', labelpad=-55, fontweight='normal')))
electricity_index.save_image(os.path.join(nepal.output_directory, 'Electricity_priority_areas.pdf'), 
                             cmap='magma', ticks=[0, 0.5, 1], tick_labels=['Low', 'Medium', 'High'],
                             admin_layer=nepal.mask_layer, figsize=(13,9), dpi=300,
                             scale_bar=scale_bar_prop, north_arrow=north_arrow_prop,
                             colorbar_kwargs=dict(orientation='horizontal', x=0.05, y=0.1, width=0.3, height=0.05,
                                            title_prop=dict(loc='left', labelpad=-55, fontweight='normal')))

In [ ]:
histogram(model=nepal_onstove, index=electricity_index, name='Electricity',
         labels=labels, cmap=cmap, output_folder=nepal.output_directory)

In [ ]:
goal_elec = nepal_onstove.gdf['Households'].sum() * 0.25
goal_biogas = 200000
goal_biomass = 500000

prioritize_households(nepal_onstove, electricity_index, ['Electricity', 'Mini_Grids'], goal_elec)
prioritize_households(nepal_onstove, biogas_index, 'Biogas', goal_biogas)
prioritize_households(nepal_onstove, biomass_ics_index, 'Biomass_ICS_T3', goal_biomass)

In [ ]:
prioritize_households_2(nepal_onstove, biomass_ics_index, 'Biomass_ICS_T3', goal_biomass)

In [ ]:
nepal_onstove.extract_lives_saved(column='Prioritized_hh')
nepal_onstove.extract_health_costs_saved(column='Prioritized_hh')
nepal_onstove.extract_time_saved(column='Prioritized_hh')
nepal_onstove.extract_opportunity_cost(column='Prioritized_hh')
nepal_onstove.extract_reduced_emissions(column='Prioritized_hh')
nepal_onstove.extract_emissions_costs_saved(column='Prioritized_hh')
nepal_onstove.extract_investment_costs(column='Prioritized_hh')
nepal_onstove.extract_fuel_costs(column='Prioritized_hh')
nepal_onstove.extract_om_costs(column='Prioritized_hh')
nepal_onstove.extract_salvage(column='Prioritized_hh')

In [ ]:
# Calculating subsidies
exchange_rate = 123.9838 # NPR/USD
## LPG
subsidy_lpg = 0
if scenario != 'Stated_policies':
    subsidy_hh_lpg = nepal_onstove.techs['LPG'].energy / nepal_onstove.techs['LPG'].energy_content / 14.2 * 549.875 / exchange_rate
    subsidy_lpg = nepal_onstove.gdf.loc[nepal.gdf['Prioritized_hh'] == 'LPG', 'Households'].sum() * subsidy_hh_lpg / 1000000000

## Biomass ICS
subsidy_ics = (nepal_onstove.gdf.loc[nepal_onstove.gdf['Prioritized_hh']=='Biomass_ICS_T3', 'Households'] * 
               nepal_onstove.techs['Biomass_ICS_T3'].inv_cost).sum() / 1000000000
if scenario == 'Social_optimum':
    subsidy_ics *= 0.5

## Biogas
eco_zones = VectorLayer(path=os.path.join('..', '1. Data', 'Ecological zones', 'Ecological zones.geojson'))
eco_zones.reproject(3857)
zones = nepal_onstove.gdf[['Households', 'geometry']].sjoin(eco_zones.data[['zone', 'geometry']], how='left')
nepal_onstove.gdf['eco_zone'] = zones['zone']
subsidy_biogas = nepal_onstove.gdf.loc[(nepal_onstove.gdf['Prioritized_hh'] == 'Biogas') & 
                               (nepal_onstove.gdf['eco_zone'] == 'Terai'), 'Households'].sum() * 193.5736766
subsidy_biogas += nepal_onstove.gdf.loc[(nepal_onstove.gdf['Prioritized_hh'] == 'Biogas') & 
                                (nepal_onstove.gdf['eco_zone'] == 'Hills'), 'Households'].sum() * 262.1310203
subsidy_biogas += nepal_onstove.gdf.loc[(nepal_onstove.gdf['Prioritized_hh'] == 'Biogas') & 
                                (nepal_onstove.gdf['eco_zone'] == 'Mountain'), 'Households'].sum() * 314.5572244
subsidy_biogas /= 1000000000

## Electricity
subsidy_electricity = 0
if scenario in ['Stated_policies']:
    increment = 549.875 / (exchange_rate * 14.2) # USD per kg
    
    increment_per_MJ = increment / nepal_onstove.techs['LPG'].energy_content # in USD/MJ
    el_sub_factor = 0.5
    subsidy_per_kwh = increment_per_MJ * nepal_onstove.techs['Electricity'].energy_content * el_sub_factor # in USD/kWh
    
    kwh_grid = nepal_onstove.techs['Electricity'].energy / nepal_onstove.techs['Electricity'].energy_content
    kwh_minigrid = nepal_onstove.techs['Mini_Grids'].energy / nepal_onstove.techs['Mini_Grids'].energy_content
    
    subsidy_electricity = nepal_onstove.gdf.loc[nepal_onstove.gdf['Prioritized_hh'] == 'Electricity', 'Households'].sum() * kwh_minigrid * subsidy_per_kwh
    subsidy_electricity += nepal_onstove.gdf.loc[nepal_onstove.gdf['Prioritized_hh'] == 'Mini_Grids', 'Households'].sum() * kwh_minigrid * subsidy_per_kwh
    subsidy_electricity /= 1000000000
    
all_subsidies = subsidy_lpg + subsidy_ics + subsidy_biogas + subsidy_electricity

# Calculating total system cost
summary = nepal_onstove.summary(total=True, pretty=False, variable='Prioritized_hh', remove_none=True)
total_costs = (summary.loc['total', 'investment_costs'] + summary.loc['total', 'fuel_costs'] + 
               summary.loc['total', 'om_costs'] - summary.loc['total', 'salvage_value']) / 1000

if scenario != 'Social_optimum':
    total_costs += all_subsidies

In [ ]:
subsidies_df = pd.DataFrame({'LPG subsidies (BUS$)': [subsidy_lpg],
                             'Biomass ICS T3 subsidies (BUS$)': [subsidy_ics],
                             'Biogas subsidies (BUS$)': [subsidy_biogas],
                             'Electricity subsidies (BUS$)': [subsidy_electricity]},
                            index=['Subsidies prioritized'])
subsidies_df.to_csv(os.path.join(nepal.output_directory, 'Subsidies_prioritized.csv'))
subsidies_df

In [ ]:
cmap = {"Biomass ICS T3": '#57365A', "LPG": '#40bdbc', "Traditional Biomass": '#673139',
        "Biogas": '#73af48', "Biogas and Biomass ICS T2": "#F6029E", "Biomass ICS T2": '#B6195E',
        "Biogas and LPG": "#156e5e",  "Biogas and Traditional Biomass": "#266AA6",
        "Biogas and Biomass ICS T3": "#bd57b6", "Biogas and Electricity": "#484673",
        "Electricity": '#d14a36', "Electricity and Biomass ICS T2": "#4D7126",
        "Electricity and LPG": "#dec610", "Electricity and Traditional Biomass": "#FFC107",
        "Electricity and Biomass ICS T3": "#1E88E5", "Electricity and Biogas": "#e17c05",
        "Mini Grids": "#ff7678", "Electricity and Mini Grids": "#ffd9df",
        "Biogas and Mini Grids": "#ffd9df", 'None': '#f1f1f1', 
        'Biogas (partially)': '#6d9991', 'Biomass ICS T3 (partially)': '#e4b7e8',
        'Electricity (partially)': '#e37a6b'}

labels = {"Biogas and Electricity": "Electricity and Biogas",
          "Electricity and None": 'Electricity (partially)',
          "None and Electricity": 'Electricity (partially)',
          "Biogas and None": 'Biogas (partially)',
          "None and Biogas": 'Biogas (partially)',
          "None and Biomass ICS T3": 'Biomass ICS T3 (partially)',
          "Biomass ICS T3 and Biomass ICS T3": 'Biomass ICS T3',
          "Biomass ICS T3 and Biogas": "Biogas and Biomass ICS T3"}

scale_bar_prop = dict(size=100000, style='double', textprops=dict(size=11), location=(0.5, 0.02),
                      linekw=dict(lw=1, color='black'), extent=0.01, loc='lower center')
north_arrow_prop = dict(size=30, location=(0.92, 0.92), linewidth=0.5)

nepal_onstove.plot('Prioritized_hh', cmap=cmap,
                   labels=labels,
                   legend_position=(0.65, 0.95), 
                   figsize=(13, 9),  
                   legend_title='Prioritized areas',
                   stats=True, 
                   scale_bar=scale_bar_prop, north_arrow=north_arrow_prop,
                   stats_kwargs={'extra_stats': {'Total system cost': f'{total_costs:.2f} BUS$',
                                 'Subsidies': f'{all_subsidies:.2f} BUS$'}, 
                                 'fontsize': 14, 'stats_position': (0, 0.28), 'pad': 2, 'sep': 0})

In [ ]:
plt.clf()

# Draw histogram
max_val = nepal_onstove.gdf['relative_wealth'].max()
min_val = nepal_onstove.gdf['relative_wealth'].min()
binwidth = (max_val - min_val) * 0.05

fig = nepal_onstove.plot_distribution(type='histogram', fill='Prioritized_hh', variable='relative_wealth', 
                                       groupby='None', cmap=cmap, labels=labels, 
                                       hh_divider=1000, y_title='Households (k)', font_args=dict(color='black', size=11),
                                       height=2.5, width=4, kwargs=dict(binwidth=binwidth, alpha=0.8, size=0.2),
                                       theme_name='minimal', quantiles=True)

# Add quantile lines
q1, q3 = weighted_percentile(a=nepal_onstove.gdf['relative_wealth'].values, q=(25, 75), weights=nepal_onstove.gdf['Households'].values)

# get figure to annotate
ax = fig.axes[0] # get the matplotlib axis (may be more than one if faceted)

# annotate quantiles
trans = ax.get_xaxis_transform()
ax.annotate('Q1', xy=(q1, 1.05), xycoords=trans,
            horizontalalignment='center',
            color='#4D4D4D', weight="bold")
ax.annotate('Q3', xy=(q3, 1.05), xycoords=trans,
            horizontalalignment='center',
            color='#4D4D4D', weight="bold")

# annotate percentages
dff = nepal_onstove.gdf.groupby('Prioritized_hh')['Households'].sum()
dff.sort_values(inplace=True)
total = dff.sum()
## Dictionaries to set the possition of the arrows and text in the histogram
y_arrow_dict = {'Electricity': 125, 'LPG': 180, 'Biogas': 10, 'Biomass_ICS_T3': 90, 'Mini_Grids': 0}
y_text_dict= {'Electricity': 340, 'LPG': 300, 'Biogas': 250, 'Biomass_ICS_T3': 400, 'Mini_Grids': 100}
x_text_dict = {'Electricity': 1.5, 'LPG': 2, 'Biogas': -1.2, 'Biomass_ICS_T3': -1, 'Mini_Grids': -1.3}

## Draw arrows and text
for tech in dff.index:
    if tech != 'None':
        group = nepal_onstove.gdf.loc[nepal_onstove.gdf['Prioritized_hh']==tech]
        hh = group['Households'].sum()
        share = hh/total
        if share > 0.01:
            x = weighted_percentile(a=group['relative_wealth'].values, q=(50), weights=group['Households'].values)
            y_arrow = y_arrow_dict[tech]
            y_text = y_text_dict[tech]
            x_text = x_text_dict[tech]

            ax.annotate(xy=(x, y_arrow),
                        xytext=(x_text, y_text), text=f'{share:.0%}',
                        horizontalalignment='right', verticalalignment='center',
                        arrowprops={'arrowstyle': '-', 'color': '#4D4D4D', 'lw': 0.5},
                        color='#4D4D4D')
        
fig.set_size_inches(4, 2.5)
fig.savefig(os.path.join(nepal.output_directory, 'tech_histogram_prioritized.pdf'), bbox_inches='tight', transparent=True)
fig

In [ ]:
q1, q3 = weighted_percentile(a=nepal_onstove.gdf['relative_wealth'].values, q=(25, 75), weights=nepal_onstove.gdf['Households'].values)
nepal_onstove.gdf.loc[nepal_onstove.gdf['relative_wealth']>q3, 'wealth_q'] = '>Q3'
nepal_onstove.gdf.loc[nepal_onstove.gdf['relative_wealth']<=q3, 'wealth_q'] = 'Q1-Q3'
nepal_onstove.gdf.loc[nepal_onstove.gdf['relative_wealth']<=q1, 'wealth_q'] = 'Q1'
total = nepal_onstove.gdf.groupby(['Prioritized_hh']).agg({'Calibrated_pop': 'sum'})
dff = nepal_onstove.gdf.groupby(['Prioritized_hh', 'wealth_q']).agg({'Calibrated_pop': 'sum'}).reset_index()
dff['percentage'] = dff['Calibrated_pop'] / dff.reset_index()['Prioritized_hh'].map(total['Calibrated_pop'])
dff

In [ ]:
nepal_onstove.output_directory = nepal.output_directory
nepal_onstove.to_image('Prioritized_hh', name='Prioritized_hh.pdf', cmap=cmap, 
                       figsize=(13, 9), dpi=300, 
                       labels=labels, legend=False, legend_title='Maximum benefit\ncooking technology', 
                       rasterized=True, stats=True, 
                       scale_bar=scale_bar_prop, north_arrow=north_arrow_prop,
                       stats_kwargs={'extra_stats': {'Total system cost': f'{total_costs:.2f} BUS$',
                                     'Subsidies': f'{all_subsidies:.2f} BUS$'}, 
                                     'fontsize': 14, 'stats_position': (0, 0.28), 'pad': 2, 'sep': 0})

In [ ]:
nepal_onstove.plot_costs_benefits(labels=labels, height=70/24.5, width=5.5,
                                  save_as='cost_benefits_prioritized', 
                                  variable='Prioritized_hh')

In [ ]:
clean_current = (nepal_onstove.gdf['Calibrated_pop'] * nepal_onstove.clean_cooking_access).sum()

In [ ]:
current_access = clean_current / nepal_onstove.gdf['Calibrated_pop'].sum()
print('Current clean cooking access: {:.1%}'.format(current_access))

In [ ]:
nepal_onstove.gdf['Clean_pop'] = np.array(nepal_onstove.gdf['Calibrated_pop'] * nepal_onstove.clean_cooking_access)
dff = nepal_onstove.gdf.groupby(['Prioritized_hh'])[['Calibrated_pop', 'Clean_pop']].sum().reset_index()
clean =(dff['Prioritized_hh'].isin(['Electricity', 'Biogas']))
none = (dff['Prioritized_hh']=='None')
total = dff.loc[clean, 'Calibrated_pop'].sum() + dff.loc[none, 'Calibrated_pop'].sum()
new_access = (dff.loc[clean, 'Calibrated_pop'].sum() + (dff.loc[none, 'Clean_pop']).sum()) / total
print('New clean cooking access: {:0.1%}'.format(new_access))

In [ ]:
print('Overall clean cooking access improvement: {:0.1%}'.format(new_access - current_access))

In [ ]:
with open(os.path.join(output_directory, 'access_results.txt'), 'w') as file:
    file.write('Current clean cooking access: {:.1%}\n'.format(current_access))
    file.write('New clean cooking access: {:0.1%}\n'.format(new_access))
    file.write('Overall clean cooking access improvement: {:0.1%}'.format(new_access - current_access))